In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import datetime
from pathlib import Path
from IPython.display import display


from utils import (
    load_dataset,
    get_images,
    transform_points,
    get_densities,
    predict,
    calculate_errors,
    get_result_dict,
    SCALING_FACTOR,
    plot_setup, 
    plot_setup_noised,
    plot_predictions,
    add_outlier,
    get_numpy_arr
)

In [ ]:
fov_pts_0026 = [
    [8.009250710227207, 3014.859952059659],
    [39.4274325284091, 1184.7508611505682],
    [1979.5001598011354, 666.3508611505681],
    [2270.1183416193176, 909.8417702414772],
    [2277.972887073863, 1231.8781338778408],
    [2513.609250710226, 1271.1508611505683],
    [3102.700159801135, 540.678133877841],
    [4029.536523437499, 898.0601917613635],
    [4021.6819779829534, 3010.932919034091]
]

fov_pts_0029 = [
    [0, 3000],
    [0, 630], 
    [2341, 364],
    [2375, 528],
    [2142, 892],
    [2281, 1283],
    [2737, 1287], 
    [2848, 995],
    [2909, 314],
    [3365, 256],
    [3630, 312], 
    [4000, 491], 
    [4000, 3000]
]
fov_pts_0032 = [
    [1034, 3000],
    [1261, 219],
    [1444, 161],
    [2249, 109],
    [2853, 1178],
    [3865, 3000]
]

position_0026 = [2719.51263607, 1594.32214866, 551.05376955]
position_0029 = [2437.49317957, 1477.44318179, 545.80213301]
position_0032 = [2418.68338479, 1487.61216757, 555.07062488]

In [ ]:
def get_predicted_fov_pts(img_name, fov_pts):
    pv_img = img_name
    tv_img = 'IMG_00'
    reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv = load_dataset(pv_img, tv_img)
    reference_pts_pv_arr, reference_pts_tv_arr, validation_pts_pv_arr, validation_pts_tv_arr = transform_points(
            reference_pts_pv, reference_pts_tv, validation_pts_pv, validation_pts_tv)
    # Calculate Homography
    h, _ = cv2.findHomography(
        reference_pts_pv_arr,
        reference_pts_tv_arr,
        # method = cv2.RANSAC,
        method = 0,
    )
    h_inv = np.linalg.inv(h)

    fov_pts_pv_arr = [[pt[0], pt[1], 1] for pt in fov_pts]
    fov_pts_pv_arr = np.array(fov_pts_pv_arr)
    predicted_fov_pts = predict(fov_pts_pv_arr, h)
    return predicted_fov_pts
predicted_fov_pts_0026 = get_predicted_fov_pts('IMG_01', fov_pts_0026)
predicted_fov_pts_0029 = get_predicted_fov_pts('IMG_02', fov_pts_0029)
predicted_fov_pts_0032 = get_predicted_fov_pts('IMG_03', fov_pts_0032)

In [ ]:
pv_img = 'IMG_01'
tv_img = 'IMG_00'

_, img_tv = get_images(pv_img, tv_img)

In [ ]:
img_tv_g = cv2.imread('data/images/IMG_00.JPG', cv2.IMREAD_GRAYSCALE)
# img_tv_g = cv2.cvtColor(img_tv, cv2.COLOR_RGB2GRAY)
plt.imshow(img_tv_g, cmap="gray", alpha=0.5)
def plot_fov(fov_pts,c):
    plt.fill(fov_pts[:,0], fov_pts[:,1], color=c, alpha=0.6)
    plt.scatter(fov_pts[:,0], fov_pts[:,1], color=c, marker='o', s=10)

def plot_position(position,c):
    plt.scatter(position[0], position[1], marker='h', s=200, color=c, alpha=1.0, ec=c)
    
plot_fov(predicted_fov_pts_0026, '#440154FF')
plot_fov(predicted_fov_pts_0029, '#2A788EFF')
plot_fov(predicted_fov_pts_0032, '#7AD151FF')
plot_position(position_0026, '#440154FF')
plot_position(position_0029, '#2A788EFF')
plot_position([position_0032[0]+90, position_0032[1]-20], '#7AD151FF')
plt.axis('off')
plt.tight_layout()
plt.savefig('output/overview.png', dpi=300)
plt.show()